In [14]:
import os
import psycopg2
import psycopg2.extras
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder


# https://stackoverflow.com/questions/43515877/should-binary-features-be-one-hot-encoded


# Database connection setup
def get_db_connection():
    connection = psycopg2.connect(
        database="postgres",
        user="postgres",
        password="HeliosChenThomasBuckley",
        host="db.fxxqwotagugztamftphi.supabase.co",
        port="5432",
    )
    
    print('connected')
    return connection



In [15]:
def fetch_profiles():
    conn = get_db_connection()
    with conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor) as cur:
        cur.execute('SELECT * FROM "Profile" LIMIT 100;')
        profiles = cur.fetchall()
    conn.close()
    return profiles

df = pd.DataFrame(fetch_profiles())


connected


In [17]:
categorical_columns = ['body_type', 'education', 'pets', 'diet', 'offspring', 'job', 'smokes', 'drinks', 'religion',]

# One-hot encoding for categorical data with multiple value possibilities
encoder = OneHotEncoder(sparse=False)
encoded_data = encoder.fit_transform(df[categorical_columns])

# Binary encoding for the binary data (smokes, drinks, religion)
# df['smokes'] = df['smokes'].map({'no': 0, 'yes': 1})
# df['drinks'] = df['drinks'].map({'no': 0, 'yes': 1})
# df['religion'] = df['religion'].map({'religious': 0, 'not religious': 1})

# print(df['smokes'].unique())
# print(df['drinks'].unique())
# print(df['religion'].unique())




/Users/thomasbuckley/Documents/CS/MSC_PROJECT/data_cleaning/venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
similarity_matrix = cosine_similarity(encoded_data)

# Convert to DataFrame for better readability (optional)
similarity_df = pd.DataFrame(similarity_matrix, index=df.index, columns=df.index)

similarity_df


       id  user_id smokes orientation body_type      diet drinks   education  \
0   35798     6507     no    straight  athletic  anything    yes  university   
1   34008     4717     no    straight  athletic  anything    yes  university   
2   36588     7297     no    straight      thin  anything    yes  university   
3   33592     4301    yes    straight      thin  anything    yes  university   
4   34628     5337     no    straight     large  anything     no  university   
..    ...      ...    ...         ...       ...       ...    ...         ...   
95  29464      173    yes    straight  athletic  anything    yes  university   
96  29474      183     no    straight     large  anything    yes  university   
97  29478      187    yes    straight  athletic  anything    yes  university   
98  31433     2142    yes         gay   average  anything    yes  university   
99  29484      193     no    straight  athletic  anything     no        none   

   height                       job  ..

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.777778,0.555556,0.444444,0.444444,0.666667,0.555556,0.444444,0.444444,0.333333,...,0.222222,0.333333,0.444444,0.777778,0.666667,0.444444,0.444444,0.444444,0.444444,0.333333
1,0.777778,1.000000,0.555556,0.555556,0.333333,0.666667,0.666667,0.444444,0.555556,0.444444,...,0.333333,0.333333,0.444444,0.666667,0.555556,0.555556,0.666667,0.555556,0.666667,0.333333
2,0.555556,0.555556,1.000000,0.777778,0.444444,0.666667,0.555556,0.444444,0.444444,0.555556,...,0.444444,0.555556,0.666667,0.555556,0.555556,0.555556,0.555556,0.555556,0.444444,0.444444
3,0.444444,0.555556,0.777778,1.000000,0.333333,0.666667,0.555556,0.444444,0.666667,0.777778,...,0.444444,0.333333,0.444444,0.333333,0.444444,0.555556,0.555556,0.555556,0.666667,0.333333
4,0.444444,0.333333,0.444444,0.333333,1.000000,0.444444,0.666667,0.666667,0.555556,0.444444,...,0.444444,0.666667,0.555556,0.444444,0.777778,0.333333,0.555556,0.333333,0.222222,0.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.444444,0.555556,0.555556,0.555556,0.333333,0.444444,0.555556,0.444444,0.666667,0.666667,...,0.111111,0.444444,0.555556,0.666667,0.555556,1.000000,0.666667,1.000000,0.555556,0.444444
96,0.444444,0.666667,0.555556,0.555556,0.555556,0.777778,0.666667,0.777778,0.555556,0.666667,...,0.444444,0.444444,0.555556,0.555556,0.555556,0.666667,1.000000,0.666667,0.555556,0.444444
97,0.444444,0.555556,0.555556,0.555556,0.333333,0.444444,0.555556,0.444444,0.666667,0.666667,...,0.111111,0.444444,0.555556,0.666667,0.555556,1.000000,0.666667,1.000000,0.555556,0.444444
98,0.444444,0.666667,0.444444,0.666667,0.222222,0.555556,0.444444,0.333333,0.555556,0.666667,...,0.222222,0.333333,0.555556,0.333333,0.333333,0.555556,0.555556,0.555556,1.000000,0.111111


In [20]:
def recommend_for_user(user_index, top_n=5):
    # Get similarity scores for the user
    user_similarity_scores = similarity_df[user_index]

    # Sort the scores and fetch top N indices
    top_users_indices = user_similarity_scores.sort_values(ascending=False).index[1:top_n+1]

    # Fetch the top recommended users/items
    return df.loc[top_users_indices]

recommendations = recommend_for_user(0, top_n=5)
print(df.loc[0])
print(recommendations.iloc[0])


          0         1         2         3         4         5         6   \
0   1.000000  0.777778  0.555556  0.444444  0.444444  0.666667  0.555556   
1   0.777778  1.000000  0.555556  0.555556  0.333333  0.666667  0.666667   
2   0.555556  0.555556  1.000000  0.777778  0.444444  0.666667  0.555556   
3   0.444444  0.555556  0.777778  1.000000  0.333333  0.666667  0.555556   
4   0.444444  0.333333  0.444444  0.333333  1.000000  0.444444  0.666667   
..       ...       ...       ...       ...       ...       ...       ...   
95  0.444444  0.555556  0.555556  0.555556  0.333333  0.444444  0.555556   
96  0.444444  0.666667  0.555556  0.555556  0.555556  0.777778  0.666667   
97  0.444444  0.555556  0.555556  0.555556  0.333333  0.444444  0.555556   
98  0.444444  0.666667  0.444444  0.666667  0.222222  0.555556  0.444444   
99  0.333333  0.333333  0.444444  0.333333  0.555556  0.333333  0.555556   

          7         8         9   ...        90        91        92        93  \
0   0.